In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Lire le graphe depuis le fichier

In [ ]:
def lire_graphe(fichier):
    """
    Lit un graphe à partir d'un fichier et le retourne sous forme de liste d'adjacence.
    :param fichier: Chemin vers le fichier contenant le graphe.
    :return: Dictionnaire représentant la liste d'adjacence du graphe.
    """
    graphe = {}
    with open(fichier, 'r') as f:
        for ligne in f:
            # Suppression des espaces inutiles et saut de ligne
            ligne = ligne.strip()

            #u, v = map(int, ligne.split(","))   # Split par ","
            u, v = map(int, ligne.split())  # Split par espace
            
            # Ajouter l'arête dans les deux sens pour un graphe non orienté
            graphe.setdefault(u, []).append(v)
            graphe.setdefault(v, []).append(u)
    return graphe


In [ ]:
def afficher_graphe(fichier):
    edges = pd.read_csv(fichier, sep=" ", header=None, names=["Source", "Target"])

    # Create the graph
    graph = nx.from_pandas_edgelist(edges, source="Source", target="Target")

    # Layout for better node positioning
    pos = nx.spring_layout(graph, seed=69)  # Spring layout for a more aesthetic layout

    # Draw the graph with improved parameters
    plt.figure(figsize=(12, 8))  # Larger figure for better visualization
    nx.draw(
        graph, 
        pos, 
        with_labels=True, 
        node_size=500,           # Larger nodes for better visibility
        node_color="lightblue",  # Change node color
        font_size=10,            # Adjust font size
        font_color="black",      # Label color
        font_weight="bold",      # Bold font for labels
        edge_color="gray",       # Edge color
        #alpha=0.8,               # Slight transparency for edges
        linewidths=1.5           # Edge line width
    )
    #plt.title("Visual Cortex Graph", fontsize=16)  # Add a title with larger font size
    plt.show()

In [ ]:
def visualiser_graphe_par_etape(graphe, chemin):
    """
    Affiche le graphe à chaque étape avec les sommets brûlés et non brûlés.

    :param graphe: Dictionnaire représentant le graphe (liste d'adjacence).
    :param chemin: Liste des sommets choisis comme sources de feu.
    """
    # Créer un objet graphe NetworkX
    G = nx.Graph()
    for sommet, voisins in graphe.items():
        for voisin in voisins:
            G.add_edge(sommet, voisin)

    # Initialisation : tous les sommets sont non brûlés
    couleurs = {sommet: "lightgrey" for sommet in graphe.keys()}

    # Afficher le graphe à chaque étape
    for etape, source in enumerate(chemin, start=1):
        # Mettre à jour la couleur du sommet source en rouge
        couleurs[source] = "red"

        # Trouver les voisins à l'étape actuelle
        voisins = graphe[source]
        for voisin in voisins:
            if couleurs[voisin] == "lightgrey":  # Ne colorer que les sommets non brûlés
                couleurs[voisin] = "yellow"

        # Dessiner le graphe avec NetworkX
        pos = nx.spring_layout(G, seed=69)
        plt.figure(figsize=(12, 8))
        nx.draw(
            G,
            pos,
            with_labels=True,
            node_color=[couleurs[node] for node in G.nodes()],
            node_size=500,
            font_size=10,
            font_color="black",
            font_weight="bold",
            edge_color="gray",
            linewidths=1.5,
        )
        plt.title(f"Étape {etape}: Source {source}")
        plt.show()